In [26]:
from pyevtk.hl import gridToVTK 
import numpy as np
%cd /home/gida3pc/Documentos/MODFLOW
#Abrir nuestro archivo
f = open('calibracion_10sm.dis').read().splitlines()

/home/gida3pc/Documentos/MODFLOW


In [27]:
#Identificar cantidad de NLAY, NROW, NCOL
paramdisc = map(int,f[6].split(' ')[1:4])
paramdisc

[10, 137, 153]

In [28]:
#Identificar origen OJO, la elev inferior se pone a mano
origen = map(np.float64,f[1].translate(None,"# Upper left corner: (").translate(None,")").split(','))
origen.append(3200)
origen

[782367.67015973397, 9242294.0654836092, 3200]

In [29]:
itemsinternos = []
for item in f:
    if item.startswith('INTERNAL 1.0') == True or item.startswith('CONSTANT') == True:
        itemsinternos.append(item)
itemsinternos

['INTERNAL 1.0 (FREE)    12 # DELR',
 'INTERNAL 1.0 (FREE)    12 # DELC',
 'INTERNAL 1.0 (FREE)    12 # TOP',
 'INTERNAL 1.0 (FREE)    12 # BOTM, Aluvial/RocMet Layer 1',
 'INTERNAL 1.0 (FREE)    12 # BOTM, Aluvial/RocMet Layer 2',
 'INTERNAL 1.0 (FREE)    12 # BOTM, Acuif_Sup Layer 1',
 'INTERNAL 1.0 (FREE)    12 # BOTM, Acuif_Sup Layer 2',
 'INTERNAL 1.0 (FREE)    12 # BOTM, Acuif_Sup Layer 3',
 'INTERNAL 1.0 (FREE)    12 # BOTM, Acuif_Med Layer 1',
 'INTERNAL 1.0 (FREE)    12 # BOTM, Acuif_Med Layer 2',
 'INTERNAL 1.0 (FREE)    12 # BOTM, Acuif_Med Layer 3',
 'INTERNAL 1.0 (FREE)    12 # BOTM, Acuif_Inf Layer 1',
 'CONSTANT    3.200000000000E+003  # BOTM, Acuif_Inf Layer 2']

In [30]:
indexinterfases = []
for item in itemsinternos:
    indexinterfases.append(f.index(item))
print indexinterfases,

[8, 25, 40, 2233, 4426, 6619, 8812, 11005, 13198, 15391, 17584, 19777, 21970]


In [31]:
def explotar(interfaseinicio, interfasefinal,arreglo):
    for variable in f[interfaseinicio+1:interfasefinal]:
        suma = variable.split('  ')
        for dato in suma:
            arreglo.append(float(dato))

In [32]:
iniciox, inicioy = [float(origen[0])],[float(origen[1])]
iniciox

[782367.670159734]

In [33]:
#matrices de ceros
arr_col = np.zeros([paramdisc[1]+1,paramdisc[2]+1,paramdisc[0]+1])
arr_fil = np.zeros([paramdisc[1]+1,paramdisc[2]+1,paramdisc[0]+1])
arr_elev = np.zeros([paramdisc[1]+1,paramdisc[2]+1,paramdisc[0]+1])
arr_headpt = np.zeros([paramdisc[1]+1,paramdisc[2]+1,paramdisc[0]+1])
arr_head = np.zeros([paramdisc[1],paramdisc[2],paramdisc[0]])

#importamos los inicios
iniciox, inicioy = [float(origen[0])],[float(origen[1])]

#for i in indexinterfases[0:1]: 
explotar(indexinterfases[0],indexinterfases[1],iniciox)
explotar(indexinterfases[1],indexinterfases[2],inicioy)

arr_x = np.cumsum(np.array(iniciox))
arr_y = np.cumsum(np.array(inicioy))

In [34]:
for i in range(paramdisc[2] + 1):
    arr_col[:,i,:] = round(arr_x[i],0)

In [35]:
for j in range(paramdisc[1] + 1):
    arr_fil[j,:,:] = round(arr_y[j],0)

In [36]:
for i in np.arange(paramdisc[0]):
    inicioz = []
    explotar(indexinterfases[i+2],indexinterfases[i+3],inicioz)
    arr_z = np.array(inicioz)
    a = arr_z.reshape(paramdisc[1],paramdisc[2])
    arr_elev[:,:,paramdisc[0]-i].fill(a.mean())
    arr_elev[0:-1,0:-1,paramdisc[0]-i] = a

arr_elev[:,:,0].fill(origen[2])

In [37]:
def explotarh(interfaseinicio, interfasefinal,arreglo):
    for variable in h[interfaseinicio+1:interfasefinal]:
        suma = variable.split('  ')
        for dato in suma:
            arreglo.append(float(dato))

In [38]:
h = open('calibracion_10sm.fhd').read().splitlines()
headsinternos = []
for item in h:
    if item.startswith('     1    1   1.000000E+00   1.000000E+00') == True:
        headsinternos.append(h.index(item))
headsinternos.append(len(h))
headsinternos    

[0, 2193, 4386, 6579, 8772, 10965, 13158, 15351, 17544, 19737, 21930]

In [39]:
for elevperlayer in np.arange(paramdisc[0]):
    z = []
    a = []
    arrv = []
    for linea in np.arange(headsinternos[elevperlayer]+1,headsinternos[elevperlayer+1]):
        suma = map(np.float64,h[linea].split('  ')[1:])
        for dato in suma:
            z.append(float(dato))
    arr_head[:,:,paramdisc[0]-elevperlayer-1] = np.array(z).reshape(paramdisc[1],paramdisc[2])
arr_head[arr_head <= -1.00000000e+2]= np.NAN
arr_head[-5:,1,7]

array([ nan,  nan,  nan,  nan,  nan])

In [64]:
arr_headpt[0:-1,0:-1,0:-1] = arr_head
arr_headpt[-1,:,:] = arr_headpt[-2,:,:]
arr_headpt[:,-1,:] = arr_headpt[:,-2,:]
arr_headpt[:,:,-1] = arr_headpt[:,:,-2]
#arr_headpt[-1,-1,:] = arr_headpt[0:-1,0:-1,0:-1]
#arr_headpt[-1:,-1:,16]
arr_e = arr_elev

In [66]:
gridToVTK("./structured4", arr_col, arr_fil, arr_elev, pointData = {"elev" : arr_e})

'/home/gida3pc/Documentos/MODFLOW/structured4.vts'

In [58]:
gridToVTK("./modeltop3", arr_col[:,:,9:], arr_fil[:,:,9:], arr_elev[:,:,9:], pointData = {"elev" : arr_headpt[:,:,9:]})

'/home/gida3pc/Documentos/MODFLOW/modeltop3.vts'

In [61]:
arr_col[:5,:5,9:]

array([[[ 782368.,  782368.],
        [ 782494.,  782494.],
        [ 782619.,  782619.],
        [ 782742.,  782742.],
        [ 782865.,  782865.]],

       [[ 782368.,  782368.],
        [ 782494.,  782494.],
        [ 782619.,  782619.],
        [ 782742.,  782742.],
        [ 782865.,  782865.]],

       [[ 782368.,  782368.],
        [ 782494.,  782494.],
        [ 782619.,  782619.],
        [ 782742.,  782742.],
        [ 782865.,  782865.]],

       [[ 782368.,  782368.],
        [ 782494.,  782494.],
        [ 782619.,  782619.],
        [ 782742.,  782742.],
        [ 782865.,  782865.]],

       [[ 782368.,  782368.],
        [ 782494.,  782494.],
        [ 782619.,  782619.],
        [ 782742.,  782742.],
        [ 782865.,  782865.]]])

In [60]:
arr_fil[:5,:5,9:]

array([[[ 9242294.,  9242294.],
        [ 9242294.,  9242294.],
        [ 9242294.,  9242294.],
        [ 9242294.,  9242294.],
        [ 9242294.,  9242294.]],

       [[ 9242444.,  9242444.],
        [ 9242444.,  9242444.],
        [ 9242444.,  9242444.],
        [ 9242444.,  9242444.],
        [ 9242444.,  9242444.]],

       [[ 9242594.,  9242594.],
        [ 9242594.,  9242594.],
        [ 9242594.,  9242594.],
        [ 9242594.,  9242594.],
        [ 9242594.,  9242594.]],

       [[ 9242744.,  9242744.],
        [ 9242744.,  9242744.],
        [ 9242744.,  9242744.],
        [ 9242744.,  9242744.],
        [ 9242744.,  9242744.]],

       [[ 9242894.,  9242894.],
        [ 9242894.,  9242894.],
        [ 9242894.,  9242894.],
        [ 9242894.,  9242894.],
        [ 9242894.,  9242894.]]])